One-dimensional Schrodinger equation is a linear partial differential equation that describes the quantum behaviour of a particle moving in one-dimension that governs by the potential, V(x). The equation for the one-dimensional Schrodinger equation in term of ψ"(x) is given by:
                        
                        𝜓"(𝑥) + 2m/ℏ**2 * [E - V(x)] * 𝜓(x) = 0 

where V(x) is:
             
             V(x) = ℏ**2/2m * 𝛼**2 * 𝜆 * (𝜆-1) * [1/2 - 1/cosh(𝛼𝑥)**2]

and En is:
               
               En = ℏ**2/2m * 𝛼**2 * [(𝜆 * (𝜆-1))/2 - (𝜆 - 1 - n)**2]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root_scalar

In [ ]:
#Define parameter
alpha = 1
lam = 4
hbar = 1
m = 1

In [ ]:
#Define potential of the well in term of x
def potential(x):
    return alpha**2 * lam * (lam - 1) * (0.5 - 1 / np.cosh(alpha * x)**2)

The Numerov method is used to solve the 𝜓(x) iteratively. From Schrodinger equation shown above, it becomes:
                                   
                                    𝜓"(𝑥) = -k(x)𝜓(x)

By using second-order Taylor expansion for 𝜓(x), the Numerov formula gives:

𝜓[i+1] = ((2 - (5*dx**2)/6 * k[i]) * 𝜓[i] - (1 - (dx**2)/12 * k[i-1]) * 𝜓[i-1]) / (1 + dx**2/12 * k[i+1])

Initial condition of 𝜓(x0) = psi0 and 𝜓(x1) = psi1 is used.


In [ ]:
def numerov(x, psi0, psi1, E, V):
    dx = x[1] - x[0]     # Step size
    psi = np.zeros_like(x)     # Creating array of 0
    psi[0], psi[1] = psi0, psi1     # Initial values of the wavefunction at the first two grid points

    for i in range(1, len(x) - 1):     # loops through the entire points of the grip ignoring the first and last point
        k1 = 2 * (E - V(x[i-1]))     # past iteration
        k2 = 2 * (E - V(x[i]))     # current iteration
        k3 = 2 * (E - V(x[i+1]))     # future iteration
        psi[i+1] = (2 * (1 - 5 * dx**2 / 12 * k2) * psi[i] - 
                    (1 + dx**2 / 12 * k1) * psi[i-1]) / (1 + dx**2 / 12 * k3)     # Numerov equation for iteration
    return psi     # The computed wavefunction over the entire grid

This code helps to match the computed value from the left boundary and from the right boundary. In Schrodinger equation, the wave function must be continuous at every point including the boundary. To ensure the wave function is smooth, we computed the slopes of the 𝜓(x) from the left and right side. If the function are correctly matched, the difference between both of the wave function should give 0.

In [ ]:
def matching_function(E, x, V):
    psiL = numerov(x, 0, 1e-3, E, V)     # solving numerov from the left boundary
    psiR = numerov(x[::-1], 0, 1e-3, E, V)[::-1]     # solving numerov from right boundary, x[::-1] is to reverse the calculation so that the wave funtion will move inward from the right boundary
                                                     # then it is reversed back to match the grid
    midpoint = len(x) // 2     # assuming the grid is symmetry from negative and positive side, this function will calculate the midpoint of the grid
    if psiL[midpoint] == 0 or psiR[midpoint] == 0:     # to avoid division by 0
        return np.inf
    return (psiL[midpoint] / psiL[midpoint - 1]) - (psiR[midpoint] / psiR[midpoint - 1])     # comparing the slopes from the left and right region

In [ ]:
def find_eigenvalues(x, V, n_states):
    eigenvalues = []     # storing computed eignevalues
    for n in range(n_states):
        E_min, E_max = n + 0.1, n + 1.5
        try:
            result = root_scalar(matching_function, args=(x, V), bracket=(E_min, E_max))
            if result.converged:
                eigenvalues.append(result.root)
        except ValueError:
            print(f"Warning: Unable to find eigenvalue for n = {n}")
    return eigenvalues

In [ ]:
x = np.linspace(-5, 5, 1000)  
V = lambda x: potential(x)     
n_states = 3     

In [ ]:
plt.figure(figsize=(10, 6))
for E in eigenvalues:
    psi = numerov(x, 0, 1e-3, E, V)
    psi /= np.sqrt(np.sum(psi**2) * (x[1] - x[0]))  
    plt.plot(x, psi, label=f"E = {E:.4f}")

In [ ]:
plt.plot(x, potential(x), label="Potential V(x)", linestyle="-", color="green")
plt.xlabel("x")
plt.ylabel("Psi(x) and V(x)")
plt.title("Wavefunctions and Potential")
plt.legend()
plt.grid()
plt.show()

print("Eigenvalues (E):", eigenvalues)